for each row sentence tokenize each paragraph.
for each token check to see if text is in it, if not get rid of it.


In [1]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize, sent_tokenize
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import collections

In [2]:
ps = PorterStemmer()

In [34]:
train = pd.read_csv("data/share_repurchase_paragraphs.csv")

In [4]:
train.shape

(1781, 8)

In [5]:
train.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ..."
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ..."
4,A,0001090872-17-000018,Amount Spent on Share Repurchases,repurchased,194000000,194.00,millions,"Table of Contents 2016, upon the completion ..."


In [6]:
categories = train.data_key_friendly_name.unique()

sub_df_list = list(range(len(categories)))
for ind, category in enumerate(categories):
    sub_df_list[ind] = train[train["data_key_friendly_name"] == category]

In [7]:
sub_df_list[0]

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.0,ones,"On May 28, 2015 we announced that our board ..."
5,AAL,0000006201-18-000009,Share Repurchase Authorization Date,July 2014,20140715,20100000.0,ones,4. Share Repurchase Programs and Dividends S...
12,AAP,0001158449-18-000039,Share Repurchase Authorization Date,"May 14, 2012",20120514,20100000.0,ones,The Company's stock repurchase program allows ...
25,AAPL,0000320193-17-000070,Share Repurchase Authorization Date,May 2017,20170515,20200000.0,ones,"Share Repurchase Program In May 2017, the Co..."
31,ABBV,0001551152-18-000014,Share Repurchase Authorization Date,"February 15, 2018",20180215,20200000.0,ones,"On February 15, 2018, AbbVie announced that it..."
33,ABC,0001140859-17-000047,Share Repurchase Authorization Date,November 2016,20161115,20200000.0,ones,"During the fiscal year ended September 30, 201..."
40,ABT,0001047469-18-000856,Share Repurchase Authorization Date,September 2014,20140915,20100000.0,ones,"In September 2014, the board of directors auth..."
45,ADBE,0000796343-18-000015,Share Repurchase Authorization Date,January 2017,20170115,20200000.0,ones,Stock Repurchase Program To facilitate our ...
48,ADI,0000006281-17-000144,Share Repurchase Authorization Date,August 2004,20040815,20000000.0,ones,Common Stock Repurchases The Company's commo...
53,ADM,0000007084-18-000012,Share Repurchase Authorization Date,"November 5, 2014",20141105,20100000.0,ones,"(2) On November 5, 2014, the Company's Board o..."


In [8]:
[date, auth, intent, count, amount, util, unk] = sub_df_list

In [8]:
test_p = date["paragraph_text"][0]

In [9]:
test_sent = sent_tokenize(test_p)

In [10]:
date["relevant_sentence"] = ""

/Users/maheshrao/anaconda3/envs/New_Constructs/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
date.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text,relevant_sentence
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.0,ones,"On May 28, 2015 we announced that our board ...",
5,AAL,0000006201-18-000009,Share Repurchase Authorization Date,July 2014,20140715,20100000.0,ones,4. Share Repurchase Programs and Dividends S...,
12,AAP,0001158449-18-000039,Share Repurchase Authorization Date,"May 14, 2012",20120514,20100000.0,ones,The Company's stock repurchase program allows ...,
25,AAPL,0000320193-17-000070,Share Repurchase Authorization Date,May 2017,20170515,20200000.0,ones,"Share Repurchase Program In May 2017, the Co...",
31,ABBV,0001551152-18-000014,Share Repurchase Authorization Date,"February 15, 2018",20180215,20200000.0,ones,"On February 15, 2018, AbbVie announced that it...",


In [12]:
relevant_sentence = list()
for sent in test_sent:
    if date["text"][0] in sent:
        relevant_sentence.append(sent)

In [15]:
relevant_sentence

['On May 28, 2015 we  announced that our board  of directors had approved a  new share repurchase program  (the   "2015 repurchase program").']

In [23]:
relevant_sentence = list()
missing_relevant_sentence = list()
for ind, para in enumerate(train.paragraph_text):
    if train.text[ind] not in para:
        print(train.text[ind])
        para_sent_tokens = sent_tokenize(para)
        for sent in para_sent_tokens:
            if train.text[ind] in sent:
                relevant_sentence.append(sent)
    elif train.text[ind] not in para:
        relevant_sentence.append("MISSING!")

the Company's board of directors authorized a new share repurchase program allowing the Company to purchase up to
for a total of
authorization was in addition
January 2017
the Company had repurchased a total of approximately 147.0 million shares of its common stock for approximately
In the aggregate, the Board of Directors has authorized the Company to repurchase
the Board of Directors approved a plan which authorized the repurchase of up to an additional
remained available for repurchase under the repurchase program approved by the Board of Directors
for a total cost of
Repurchase authorization remaining at December 31,
the Company's board of directors authorized the purchase of an additional
As of December 31, 2017, the Company has repurchased
our Board of Directors authorized an additional increase of
Aimco was authorized to repurchase approximately
publicly announced share repurchase authorization of (1) up to
shares of our common stock under our share repurchase program and the AS

In [10]:
len(relevant_sentence)

2034

In [11]:
train.shape

(1781, 8)

In [12]:
len(missing_relevant_sentence)

0

In [13]:
len(relevant_sentence) + len(missing_relevant_sentence)

2034

In [44]:
in_there = 0
not_in_there = 0
relevant_sentence = list()
for ind, para in enumerate(train.paragraph_text):
    if train.text[ind] in train.paragraph_text[ind]:
        in_there += 1
        para_sent_tokens = sent_tokenize(para)
        sents = list()
        for sent in para_sent_tokens:
            if train.text[ind] in sent:
                sents.append(sent)
        relevant_sentence.append(sents)
    else:
        not_in_there += 1
        relevant_sentence.append("MISSING!")

In [46]:
in_there + not_in_there

1781

In [45]:
len(relevant_sentence)

1781

In [74]:
new_df = list()
sents = list()
for ind, sent_list in enumerate(relevant_sentence):
    for sent in sent_list:
        new_df.append(train.iloc[ind])
        sents.append(sent)

In [81]:
train_sents = pd.DataFrame(new_df)
train_sents.shape

(5093, 8)

In [84]:
train_sents["relevant_sentence"] = sents

In [85]:
train_sents.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text,relevant_sentence
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ...","On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ...",The 2015 share repurchase program authorizes t...
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ...","As of October 31, 2017, we had remaining a..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ...","Table of Contents 2016, upon the completion ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ...","During the year ended October 31, 2017 ..."


In [73]:
train.iloc[9]

ticker                                                                  AAL
accession_number                                       0000006201-18-000009
data_key_friendly_name                    Amount Spent on Share Repurchases
text                                                            repurchased
data_value                                                       1600000000
reported_data_value                                                     1.6
reported_units                                                     billions
paragraph_text            During the year ended  December 31, 2017,  we ...
Name: 9, dtype: object

In [58]:
train_sent = pd.concat([train, pd.Series(relevant_sentence)], axis = 1, ignore_index = True)

In [63]:
train_sent.head()

,0,1,2,3,4,5,6,7,8
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ...","[On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ...",[The 2015 share repurchase program authorizes ...
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ...","[As of October 31, 2017, we had remaining ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ...","[Table of Contents 2016, upon the completion..."
4,A,0001090872-17-000018,Amount Spent on Share Repurchases,repurchased,194000000,194.00,millions,"Table of Contents 2016, upon the completion ...","[Table of Contents 2016, upon the completion..."


In [65]:
train.columns

Index(['ticker', 'accession_number', 'data_key_friendly_name', 'text',
       'data_value', 'reported_data_value', 'reported_units',
       'paragraph_text'],
      dtype='object')

In [66]:
train_sent.columns = ['ticker', 'accession_number', 'data_key_friendly_name', 'text',
       'data_value', 'reported_data_value', 'reported_units',
       'paragraph_text', 'relevant_sentence']

In [67]:
train_sent.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text,relevant_sentence
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ...","[On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ...",[The 2015 share repurchase program authorizes ...
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ...","[As of October 31, 2017, we had remaining ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ...","[Table of Contents 2016, upon the completion..."
4,A,0001090872-17-000018,Amount Spent on Share Repurchases,repurchased,194000000,194.00,millions,"Table of Contents 2016, upon the completion ...","[Table of Contents 2016, upon the completion..."


In [68]:
for ind, sents in enumerate(train_sent.relevant_sentence):
    if len(sent) > 1:
        print(ind)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
string = "this is a test for May 28, 2015"

In [32]:
train.text[0] in string

True